In [ ]:
# stdlib
import os
from os import environ as env
from pathlib import Path
import sys

# syft absolute
import syft as sy
from syft.service.action.action_object import AnyActionObject
from syft.service.user.user_roles import ServiceRole
from syft.util.test_helpers.email_helpers import load_users
from syft.util.test_helpers.job_helpers import create_simple_query_job
from syft.util.test_helpers.job_helpers import create_wrong_syntax_query
from syft.util.test_helpers.job_helpers import load_jobs
from syft.util.util import find_base_dir_with_tox_ini
from syft.util.util import get_caller_file_path
from syft.util.util import is_interpreter_jupyter

In [ ]:
def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(base_dir, "test_helpers")
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()

In [ ]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [ ]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

# Login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

client = sy.login(url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD)

In [ ]:
# Check if this is a new server
migration_data = client.get_migration_data()

# assert len(migration_data.store_objects[User]) == 1
# assert UserCode not in migration_data.store_objects

In [ ]:
migration_data

# Load migration data

In [ ]:
!pwd

In [ ]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
blob_path = migration_data_dir / "migration.blob"
yaml_path = migration_data_dir / "migration.yaml"

print(f"Loading migration data from {str(blob_path.resolve())}")

In [ ]:
res = client.load_migration_data(blob_path)
assert isinstance(res, sy.SyftSuccess), res.message

In [ ]:
res

In [ ]:
sy.upgrade_custom_workerpools(client, blob_path, mode="auto")

# Post migration tests

In [ ]:
users = load_users(client, path="0.9.1_notebooks/users.json")
jobs = load_jobs(users, client, filepath="0.9.1_notebooks/jobs.json")

### Check users

In [ ]:
server_users = client.users

In [ ]:
server_users[0]

In [ ]:
server_user_names = [
    user.name for user in server_users if user.role == ServiceRole.DATA_SCIENTIST
]

In [ ]:
user_names = [user.name for user in users]

In [ ]:
assert server_user_names == user_names

### Old jobs

In [ ]:
# submitted_jobs = [job for job in jobs if job.is_submitted]
reviewed_jobs = [job for job in jobs if job.admin_reviewed]
reviewed_jobs_should_succeed = [j for j in reviewed_jobs if j.should_succeed]
reviewed_jobs_should_fail = [j for j in reviewed_jobs if not j.should_succeed]

print(
    f"{len(reviewed_jobs)=}, {len(reviewed_jobs_should_succeed)=}, {len(reviewed_jobs_should_fail)=}"
)

In [ ]:
for job in reviewed_jobs_should_succeed:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method
    j = api_method(blocking=False)
    res = j.wait()

    if isinstance(res, sy.SyftError):
        raise sy.SyftException(public_message=res.message)

    result = res.get()
    job.result_as_expected = True

In [ ]:
for job in reviewed_jobs_should_fail:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method

    j = api_method(blocking=False)
    res = j.wait()
    if isinstance(res, sy.SyftError):
        job.result_as_expected = True
    else:
        raise sy.SyftException(public_message=f"failed, job didnt raise {type(j)}")

In [ ]:
expected_jobs = [job for job in jobs if job.result_as_expected]
print(f"got expected_jobs: {len(expected_jobs)} == reviewed_jobs: {len(reviewed_jobs)}")
assert len(reviewed_jobs) == len(expected_jobs)

### Use old DS to go through the flow again

In [ ]:
ds_client = users[0].client

In [ ]:
assert len(ds_client.api.services.api.api_endpoints()) == 3

In [ ]:
job = create_simple_query_job(users[0])

In [ ]:
response = ds_client.api.services.bigquery.submit_query(
    func_name=job.func_name, query=job.query
)
response

In [ ]:
assert isinstance(response, AnyActionObject)

In [ ]:
for request in client.requests:
    if request.code.service_func_name == job.func_name:
        request.approve()

In [ ]:
job_res = getattr(ds_client.code, job.func_name)(blocking=False)

In [ ]:
job_res.wait()

In [ ]:
# third party
from pandas import DataFrame

assert isinstance(job_res.result.get(), DataFrame)

In [ ]:
wrong_syntax_job = create_wrong_syntax_query(users[0])

In [ ]:
response = ds_client.api.services.bigquery.submit_query(
    func_name=wrong_syntax_job.func_name, query=wrong_syntax_job.query
)
response

In [ ]:
assert isinstance(response, AnyActionObject)

In [ ]:
for request in client.requests:
    if request.code.service_func_name == wrong_syntax_job.func_name:
        request.approve()

In [ ]:
job_res = getattr(ds_client.code, wrong_syntax_job.func_name)(blocking=False)

In [ ]:
assert isinstance(job_res.wait(), sy.SyftError)

In [ ]:
if server.server_type.value == "python":
    server.land()